# Setup and Data

In [ ]:
import pandas as pd
import pyreadstat
import numpy as np
from tabulate import tabulate

# Parameters
start_year = 2021
end_year = 2022
base_year = 2022
years=range(start_year,end_year+1)
factor = 1 # Factor in case of missing prices. 0 = ignore, 1 = assume no change in price

# Load folder names
folder_names_pathname='Data_clean/CEX_folder_names.csv'
folder_names_df = pd.read_csv(folder_names_pathname)

# CEX data folder
cex_data_folder='/Users/roykisluk/Downloads/Consumer_Expenditure_Survey/'

####################################################

# Functions

# Total consumption, monthly, across all products
def total_consumption_value(df): 
    total_consumption = 0.0
    for j in range(0, len(df)):
        total_consumption += df['omdan'][j]
    return total_consumption

# Weighting by product
def weighting(df):
    weights = pd.DataFrame(df['prodcode'].unique(), columns=['prodcode'])
    weights['weight'] = 0.0
    total_consumption = total_consumption_value(df)
    for j in range(0, len(weights)):
        weights.loc[j, 'weight'] = df[df['prodcode'] == weights.loc[j, 'prodcode']]['omdan'].sum() / total_consumption
    return weights

# Effective price paid, on average, per product
def average_price(df):
    average_prices = pd.DataFrame(df['prodcode'].unique(), columns=['prodcode'])
    average_prices['price'] = 0.0
    for j in range(0, len(average_prices)):
        average_prices.loc[j, 'price'] = (df[df['prodcode'] == average_prices.loc[j, 'prodcode']]['mehir'] / df[df['prodcode'] == average_prices.loc[j, 'prodcode']]['kamut']).mean()
    return average_prices

# Laspeyres index
def Laspeyres(df_base, df_current):
    # Create index dataframe
    index_df = pd.DataFrame(df_base['prodcode'].unique(), columns=['prodcode'])
    index_df['index'] = 0.0
    # Calculate weights and average prices
    weights = weighting(df_base)
    average_prices_base = average_price(df_base)
    average_prices_current = average_price(df_current)
    # Merge weights and average prices into index dataframe
    index_df = index_df.merge(weights, on='prodcode', how='left')
    index_df = index_df.merge(average_prices_base, on='prodcode', how='left', suffixes=('', '_base'))
    index_df = index_df.merge(average_prices_current, on='prodcode', how='left', suffixes=('_base', '_current'))
    # Calculate index
    total_index = 0.0
    # Missing prices counter
    missing_base_prices = 0
    missing_current_prices = 0
    for j in range(len(index_df)):
        # Check for missing prices
        price_current = index_df.loc[j, 'price_current']
        price_base = index_df.loc[j, 'price_base']
        if price_base == 0 or pd.isna(price_base) or np.isinf(price_base):
            index_df.loc[j, 'index'] = factor * 100
            missing_base_prices += 1
            print(f"prodcode {index_df.loc[j, 'prodcode']}: invalid price_base")
            continue
        if price_current == 0 or pd.isna(price_current) or np.isinf(price_current):
            index_df.loc[j, 'index'] = factor * 100
            missing_current_prices += 1
            print(f"prodcode {index_df.loc[j, 'prodcode']}: invalid price_current")
            continue
        # Calculate index for each product
        index_df.loc[j, 'index'] = (price_current / price_base) * 100
    # Calculate yearly price index
    for j in range(len(index_df)):
        weight = index_df.loc[j, 'weight']
        total_index += weight * index_df.loc[j, 'index']
    print(f"Missing base prices: {missing_base_prices}")
    print(f"Missing current prices: {missing_current_prices}")
    return index_df, total_index

# Load data

# Load price data for each year
dfs_prices = {}
for year in years:
    subfolder = folder_names_df.loc[folder_names_df['Year'] == year, 'Folder_Name'].values[0]
    data_prices_pathname = f"{cex_data_folder}{subfolder}/{subfolder}datayoman.sas7bdat"
    df, meta = pyreadstat.read_sas7bdat(data_prices_pathname)
    df.columns = df.columns.str.lower()
    dfs_prices[year] = df

# Calculate weights and price indexes
yearly_price_index={}
df_price_index={}
for year in years:
    df_price_index[year], yearly_price_index[year] = Laspeyres(dfs_prices[base_year], dfs_prices[year])


prodcode 305094.0: invalid price_current
prodcode 305102.0: invalid price_current
prodcode 383166.0: invalid price_current
prodcode 312116.0: invalid price_base
prodcode 312074.0: invalid price_current
prodcode 371401.0: invalid price_current
prodcode 313320.0: invalid price_current
prodcode 383513.0: invalid price_current
prodcode 371542.0: invalid price_current
prodcode 394155.0: invalid price_current
prodcode 423137.0: invalid price_current
prodcode 385104.0: invalid price_current
prodcode 375402.0: invalid price_current
prodcode 383505.0: invalid price_current
prodcode 371492.0: invalid price_current
prodcode 343319.0: invalid price_current
prodcode 372094.0: invalid price_current
prodcode 371237.0: invalid price_current
prodcode 364091.0: invalid price_current
prodcode 373183.0: invalid price_current
prodcode 342030.0: invalid price_current
prodcode 383265.0: invalid price_current
prodcode 444018.0: invalid price_current
prodcode 384107.0: invalid price_current
prodcode 383430.0: 

# Output

In [2]:
# Display yearly price index per year in tabulate
print("Yearly Price Index:")
print(tabulate(yearly_price_index.items(), headers=["Year", "Price Index"], tablefmt="grid"))

Yearly Price Index:
+--------+---------------+
|   Year |   Price Index |
+========+===============+
|   2021 |       96.4242 |
+--------+---------------+
|   2022 |      100      |
+--------+---------------+


In [15]:
# Create a new dataframe with prodcode as the first column
df_combined_index = pd.DataFrame(df_price_index[base_year]['prodcode'])

# Insert the price index for each year
for year in years:
    df_combined_index[year] = df_combined_index['prodcode'].map(df_price_index[year].set_index('prodcode')['index'])

print(df_combined_index)
print(df_combined_index[2021].mean())

     prodcode         2021   2022
0    304170.0   101.730627  100.0
1    304139.0    99.803917  100.0
2    381012.0   102.977817  100.0
3    304014.0    95.387776  100.0
4    304303.0    78.989203  100.0
..        ...          ...    ...
907  371518.0    85.000000  100.0
908  364265.0   100.000000  100.0
909  342253.0  6545.454545  100.0
910  313247.0    31.539343  100.0
911  371120.0   100.000000  100.0

[912 rows x 3 columns]
121.60858885227793


In [13]:
# Create a dictionary to store the top 10 price indexes for each year
top_10_price_indexes = {}

# Iterate through each year in df_combined_index, except for the base year
for year in years:
    if year != base_year:
        # Sort the dataframe by the current year in descending order and keep the top 10
        top_10_price_indexes[year] = df_combined_index[['prodcode', year]].sort_values(by=year, ascending=False).head(10)

# Display the top 10 price indexes for each year
for year, df_top_10 in top_10_price_indexes.items():
    print(f"Top 10 price indexes for {year}:")
    print(df_top_10)

# Create a dictionary to store the bottom 10 price indexes for each year
bottom_10_price_indexes = {}

# Iterate through each year in df_combined_index, except for the base year
for year in years:
    if year != base_year:
        # Sort the dataframe by the current year in ascending order and keep the bottom 10
        bottom_10_price_indexes[year] = df_combined_index[['prodcode', year]].sort_values(by=year, ascending=True).head(10)

# Display the bottom 10 price indexes for each year
for year, df_bottom_10 in bottom_10_price_indexes.items():
    print(f"Bottom 10 price indexes for {year}:")
    print(df_bottom_10)

Top 10 price indexes for 2021:
     prodcode         2021
909  342253.0  6545.454545
833  374207.0  3353.846154
872  371211.0   994.607020
878  423061.0   951.585977
734  393066.0   767.068273
529  333153.0   761.276596
726  383018.0   733.720245
843  391110.0   677.837838
486  397018.0   615.749365
824  383380.0   589.285714
Bottom 10 price indexes for 2021:
     prodcode      2021
654  341016.0  0.122394
795  382036.0  0.471252
844  373167.0  0.932875
402  412015.0  1.187010
815  373217.0  1.897810
884  394163.0  2.142311
865  371450.0  3.673469
793  411066.0  4.791167
849  376350.0  4.954992
814  341610.0  5.434783
